


<a> <img src='https://www.optiver.com/wp-content/uploads/2021/06/smaller-pic-ce.jpeg' /></a>
___



<a id="top"></a> <br>
#  Optiver Realized Volatility Prediction


In [ ]:
import pandas as pd
import numpy as np
import os
from joblib import Parallel, delayed
import os, sys, gc, warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 2000)

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.model_selection import KFold, GroupKFold, TimeSeriesSplit
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import r2_score

In [ ]:
path = '../input/optiver-realized-volatility-prediction/'
train_data = pd.read_csv(path+'train.csv')
test_data = pd.read_csv(path+'test.csv')
book_train_data = pd.read_parquet(path+'book_train.parquet')
book_test_data = pd.read_parquet(path+'book_test.parquet')
trade_train_data = pd.read_parquet(path+'trade_train.parquet')
trade_test_data = pd.read_parquet(path+'trade_test.parquet')
submission_data = pd.read_csv(path+'sample_submission.csv')

In [ ]:
### Code from : https://www.kaggle.com/rajgandhi/optiver-volatility-prediction-xgboost
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def get_stock_stat_df(stock_id, dataset):
    
    book = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_{dataset}.parquet/stock_id={stock_id}/')
    book['stock_id'] = stock_id
    
    # Calculate WAP
    book['wap1'] = (book.bid_price1 * book.ask_size1 + book.ask_price1 * book.bid_size1) / (book.bid_size1 + book.ask_size1)
    book['wap2'] = (book.bid_price2 * book.ask_size2 + book.ask_price2 * book.bid_size2) / (book.bid_size2+ book.ask_size2)
    
    # Log return
    book['log_return1'] = log_return(book['wap1'])
    book['log_return2'] = log_return(book['wap2'])
    
    # Bid Ask Spread
    book['bid_ask_spread1'] = abs(book['bid_price1'] - book['ask_price1'])
    book['bid_ask_spread2'] = abs(book['bid_price2'] - book['ask_price2'])
    
    book = book[~book['log_return1'].isnull()]
    book = book[~book['log_return2'].isnull()]
    
    stock_stat = book.groupby(['stock_id', 'time_id']).agg({'bid_ask_spread1':['mean'],
                                                           'bid_ask_spread2':['mean'],
                                                           'log_return1':[realized_volatility],
                                                           'log_return2':[realized_volatility]}).reset_index()
    
    # Trade
    trade =  pd.read_parquet(f'../input/optiver-realized-volatility-prediction/trade_{dataset}.parquet/stock_id={stock_id}/')
    trade['stock_id'] = stock_id
    trade_stat = trade.groupby(['stock_id', 'time_id']).agg({'price':['mean'], 'size':['sum'], 'order_count':['sum']})
    
    # Merge book and trade dataframe
    stats = stock_stat.merge(trade_stat, on=['stock_id', 'time_id'], how='left')
    
    return stats

def get_dataSet(stock_ids : list, dataType = 'train'):

    stock_stat = Parallel(n_jobs=-1)(
        delayed(get_stock_stat_df)(stock_id, dataType) 
        for stock_id in stock_ids
    )
    
    stock_stat_df = pd.concat(stock_stat, ignore_index = True)

    return stock_stat_df

In [ ]:
stock_ids = list(train_data['stock_id'].unique())
train_stock_stats_data = get_dataSet(stock_ids, 'train')
data_vol = pd.merge(train_data, train_stock_stats_data, on = ['stock_id', 'time_id'], how = 'left')
data_vol = data_vol.rename(columns={('bid_ask_spread1', 'mean'):'bid_ask_spread1', 
             ('bid_ask_spread2', 'mean'):'bid_ask_spread2',
             ('log_return1', 'realized_volatility'):'log_return1',
             ('log_return2', 'realized_volatility'):'log_return2',
             ('price', 'mean'):'Price',('size', 'sum'):'Size',
             ('order_count', 'sum'):'Order_count'})
data_vol.fillna(0, inplace=True)
X = data_vol.drop('target', axis=1)
y = data_vol['target']

In [ ]:
# TODO: make GitHub GIST
# TODO: add as dataset
# TODO: add logging with verbose

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            
## ----------------
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

def make_predictions(tr_df, tt_df, features_columns, target, var_agg, params, NFOLDS=5, kfold_mode='grouped'):
    
    X,y = tr_df[features_columns].copy(), tr_df[target].copy()
    P,P_y = tt_df[features_columns], tt_df[target].copy()
    split_groups = tr_df[var_agg]

    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))

    if kfold_mode=='grouped':
        folds = GroupKFold(n_splits=NFOLDS)
        folds_split = folds.split(X, y, groups=split_groups)

    if kfold_mode=='timeseriessplit':
        folds = TimeSeriesSplit(n_splits=NFOLDS)
        folds_split = folds.split(X, y, groups=split_groups)

    if kfold_mode=='purgedgrouptimeseriessplit':
        cv = PurgedGroupTimeSeriesSplit(
            n_splits=7,
            max_train_group_size=70,
            group_gap=2,
            max_test_group_size=5)
        folds_split = cv.split( X, y, groups=split_groups.values)
    else:
        folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
        folds_split = folds.split(X, y)

    for fold_, (trn_idx, val_idx) in enumerate(folds_split):        
        print('Fold:',fold_)
        print("len val_idx ", len(val_idx))
        print("len trn_idx ", len(trn_idx))
        
        estimator =  XGBRegressor(**params)
        x_tr, y_tr = X.iloc[trn_idx], y.iloc[trn_idx]
        x_ts, y_ts = X.iloc[val_idx], y.iloc[val_idx]
        
        estimator.fit(x_tr, y_tr,
                 eval_set=[(x_ts, y_ts)],
                 early_stopping_rounds=150,
                  verbose=False)
        pp_p = estimator.predict(P)
        predictions += pp_p/NFOLDS
        
        oof_preds = estimator.predict(X.iloc[val_idx,:])
        oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())
        
        error = rmspe(P_y, pp_p)
        print(f'RMSPE: {error}')        
        
    tt_df['prediction'] = predictions
    return tt_df, estimator

In [ ]:
SEED = 42
target_var = 'target'
var_agg = 'stock_id'

xgb_params = {
    'objective':'reg:squarederror',
    'n_jobs':-1,
 'lambda': 0.028,
 'alpha': 0.00974,
 'colsample_bytree': 0.31,
 'subsample': 0.41,
 'learning_rate': 0.02,
 'n_estimators': 1000,
 'max_depth': 6,
 'random_state': 2021,
 'min_child_weight': 289}

In [ ]:
kfold_mode='timeseriessplit'
test_predictions, xgb_model_ts = make_predictions(data_vol[:308932], data_vol[308932:], X.columns.tolist(), target_var, var_agg, xgb_params,
                                        NFOLDS=5, kfold_mode=kfold_mode)
xgb.plot_importance(xgb_model_ts)

In [ ]:
kfold_mode='purgedgrouptimeseriessplit'
test_predictions, xgb_model_purged = make_predictions(data_vol[:308932], data_vol[308932:], X.columns.tolist(), target_var, var_agg, xgb_params, 
                                        NFOLDS=5, kfold_mode=kfold_mode)
xgb.plot_importance(xgb_model_purged)

In [ ]:
%%time
test_stock_ids = list(test_data['stock_id'].unique())
test_stock_stats_data = get_dataSet(test_stock_ids, 'test')

test_data = pd.merge(test_data, test_stock_stats_data, on = ['stock_id', 'time_id'], how = 'left')
test_data.fillna(0, inplace=True)
test_data = test_data.rename(columns={('bid_ask_spread1', 'mean'):'bid_ask_spread1',
             ('bid_ask_spread2', 'mean'):'bid_ask_spread2',
             ('log_return1', 'realized_volatility'):'log_return1',
             ('log_return2', 'realized_volatility'):'log_return2',
             ('price', 'mean'):'Price',('size', 'sum'):'Size',('order_count', 'sum'):'Order_count'})
test_vol_data = test_data.drop('row_id', axis=1)

In [ ]:
pred_xgb_model = xgb_model_purged.predict(test_vol_data)
submission_data ['target'] = pred_xgb_model
submission_data.to_csv('submission.csv', index=False)

In [ ]:
submission_data